# Ejercicio práctico: Predicción de puntuación. Parte 1: Apartados de 1 a 5.

Alumno: Julián María Galindo Álvarez

DNI: 20081281E

## Introducción

Este trabajo aborda la tarea de ''Predicción de puntuación'', donde el objetivo va a ser definir una serie de modelos que puedan introducir puntuación en una entrada que carece de puntuación.

En concreto, los signos de puntuación que se van a considerar son:
 * Punto: .
 * Coma: ,
 * Punto y coma: :
 * Dos puntos: :
 * Signo de exclamación: !
 * Signo de interrogación: ?

A lo largo del trabajo se vana  definir dos modelos, uno de puntuación básica y otro basado en 4-gramas. Además se va a definir una serie de funciones para evaluar los modelos usando una función que implementaremos (VerifyPunctuation), basáda en la distancia de Levenshtein.

Este notebook contiene los apartados del 1 al 5 del trabajo, dejando el apartado 6 en el notebook TRABAJO_PLN_APARTADO_6_JULIAN_GALINDO_ALVAREZ_20091281E y el 7 en en documento TRABAJO_PLN_APARTADO_7_JULIAN_GALINDO_ALVAREZ_20091281E.pdf

Las funciones definidas en cada uno de los apartados se encuentran en los correspondientes archivos python de este directorio. En cualquier caso, este proyecto está disponible en el siguiente repositorio de github:

 * [github.com/julgalalv/PLN_Prediccion_de_puntuacion](https://github.com/julgalalv/PLN_Prediccion_de_puntuacion)
 
La estructura del repositorio es la siguiente:
 * **settings.py**: parámetros globales y creación de directorios en caso necesario. 
 * **preprocessor.py**: En este módulo se definen funciones que son necesarias para el procesamiento de los datos, necesarios para todos los apartados.
 * **punctuator_basic.py**: Funciones del Apartado 1 correspondientes a la puntuación básica.
 * **model_ngram.py**: Clase que define el modelo predictivo basado en N-gramas del Apartado 4.
 * **punctuator_ngram**: Funciones del Apartado 4 correspondientes a la puntuación basada en 4-gramas.
 * **evaluator.py**: Módulo que contiene las funciones para evaluar los módelos usados en el trabajo, esto es, los métodos correspondientes a los Apartados 2, 3 y 5.

En cuanto a los directorios tenemos:

    -root
        |-data  
        |  |-prepared
        |  |-preprocessed
        |  |-raw
        |-predicted
        |-punctuator2tf2
        |-zips

 * **data**: Contiene subdirectorios tando de datos del dataser original como procesados.
 * **raw**: Contiene los corpus de dataset originales.
 * **prepared**: Contiene archivos generados de preprocesamiento de los corpus (apartado 6)
 * **preprocessed**: Contiene archivos generados de preprocesamiento de los corpus vectorizados para el modelo de Ottokar Tilk y Tanel Alum (apartado 6).
 * **predicted**: Contiene archivos con predicciones (puntuaciones) realizadas por los modelos.
 * **punctuator2tf2**: Modelo de Ottokar Tilk y Tanel Alum (apartado 6), implementación en TensorFLow con modificaciones
 realizadas por mí para adaptalo a este trabajo.
 * **zips**: contiene el dataset y el modelo punctuator2tf2 comprimidos.
 
**NOTA**: A lo largo del trabajo voy a ir importando las funciones de cada archivo conforma hagan falta, recomiendo ir al código para consultarlas conforme se avanza en este documento.

In [1]:
import os
import settings

In [2]:
# Inicializamos parámetros globales y creamos directorios en caso necesario
settings.initialize()

In [3]:
# Definimos variables con las rutas de los archivos de corpus
TEST_RAW_PATH = os.path.join(settings.DATA_RAW_DIR,'PunctuationTask.test.en')
CHECK_RAW_PATH = os.path.join(settings.DATA_RAW_DIR,'PunctuationTask.check.en')
TRAIN_RAW_PATH = os.path.join(settings.DATA_RAW_DIR,'PunctuationTask.train.en')

## APARTADO 1: Modelo de puntuación básica. addPunctuationBasic.

Vamos a implementar un sistema que reciba una expresión como entrada (será una expresión formada solo por minúsculas y sin los signos de puntuación mencionados) y la salida será la misma expresión pero con los cambios correspondientes a la introducción de mayúsculas y signos de puntuación indicados.

Como primera versión de esta función addPunctuationBasic se implementará un modelo que
simplemente cambia la primera letra por mayúscula y añade al final del string de entrada un punto.

In [4]:
from punctuator_basic import addPunctuationBasic

Vemos que la función anterior realiza la operación correctamente con el siguiente ejemplo.

In [5]:
addPunctuationBasic('esta es una frase de prueba')

'Esta es una frase de prueba.'

## APARTADO 2: Verificación. VerifyPunctuation

Antes de definir la función verifyPuntuation vamos a definir dos funciones auxiliares que nos servirán.
* padding(list1,list2): Devuelve las los listas de entrada de forma que ambas tengan la misma longitud, añadiendo el elemento string vacío ('') a la lista de menor longitud.

* tokenizer(text): tokeniza el texto de entrada.

In [6]:
from preprocessor import padding, tokenizer

Veamos el funcionamiento de *padding* con el siguiente ejemplo:

In [7]:
list1, list2 = ['s','a'], ['1','2','3','4']
l1, l2 = padding(list1,list2)
print('Lista 1 con padding: ',l1,'\t Lista 2 con padding: ', l2)
print('Las longitudes son iguales: ', len(l1) == len(l2))

Lista 1 con padding:  ['s', 'a', '', ''] 	 Lista 2 con padding:  ['1', '2', '3', '4']
Las longitudes son iguales:  True


Veamos cómo funciona *tokenizer* con el siguiente ejemplo:

In [8]:
text_example= "Sara said: Hello, what's your name?"
print(tokenizer(text_example))

['Sara', 'said', ':', 'Hello', ',', "what's", 'your', 'name', '?']


Ahora vamos a importar la función principal de este apartado, *verifyPunctuation*.

In [9]:
from evaluator import verifyPunctuation

Utilicemos el ejemplo del documento para ver si funciona correctamente la función. Además verificamos que intercambiar check y test devuelve el mismo número de elementos (distancia de Levenshtein).

In [10]:
check_example = "Hello. What's your name?"
test_example = "Hello what's your, name?"
print('check vs test: ', verifyPunctuation(check_example,test_example))
print('test vs check: ', verifyPunctuation(test_example,check_example))

check vs test:  {('D', 1), ('S', 2), ('I', 4)}
test vs check:  {('S', 1), ('I', 1), ('D', 3)}


## APARTADO 3: Evaluación de modelos.

Implementaremos una herramienta que permita recorrer todo el corpus de test y verificación. Es decir, irá recorriendo una a una las líneas de cada fichero (que están alineadas), aplicaría sobre la frase de test el algoritmo básico de puntuación (apartado 1: *addPunctuationBasic* ) y a continuación comprobaría si el resultado es o no correcto usando la función *verifyPunctuation* del apartado 2.

En primer lugar vamos a definir una función *evaluate_example* que calcula las métricas precision y recall dado una instancia (check,test). 

In [11]:
from evaluator import evaluate_example

Comprobamos el funcionamiento de la función con el siguiente ejemplo.

* check = "Hello. What's your name?"
* test = "hello what's your name"

In [12]:
check_example = "Hello. What's your name?"
test_example = "hello what's your name"
evaluate_example(addPunctuationBasic,check_example,test_example,print_info=True)

TEST LINE: 
  hello what's your name
MODEL PUNCTUATED LINE: 
  Hello what's your name.
VALIDATION LINE: 
  Hello. What's your name? 

Modificaciones necesarias:  {('S', 0), ('S', 1), ('I', 1), ('I', 4)}
Modificaciones hechas por el modelo:  {('S', 0), ('I', 4)}
Diferencias entre modelo y validación:  {('S', 1), ('I', 1), ('S', 4)} 

n_hechas (Núm. de modificaciones hechas por el modelo):  2
n_correctas (Núm. de modificaciones correctas, i.e., 
 interseccion(hechas,necesarias) - error de sustitucion de signo):  1
n_necesarias (Núm. de modificaciones necesarias):  4 

precision (n_correctas/n_hechas):  0.5
recall (n_correctas/n_necesarias):  0.25 



(0, 0.5, 0.25, 2, 1, 4)

Notemos que la oración puntuada es "Hello what's your name.", por lo que la función de validación va a considerar como correcto el haber insertado en el token 4, lo que se observa como ('I', 4) tanto en las modificaciones necesarias como las hechas por el modelo. Al no tener información sobre el caracter, esto podría dar lugar a un falso positivo, que es corregido correctamente con el error de sustitución de signo (ver función).

Definamos ahora la función principal de este apartado, *evaluate*, que raliza el proceso anterior para distintos corpus y calcula métricas gobales.

In [13]:
from evaluator import evaluate

Evaluamos la función *addPunctuationBasic*:

In [14]:
evaluate(addPunctuationBasic, check_file_path=CHECK_RAW_PATH, test_file_path=TEST_RAW_PATH)

MÉTRICAS
precision global:  0.9482794650320423
recall global:  0.42824561955393375
F1 global:  0.5900314226893488
precision media:  0.9475733555833681
recall medio:  0.587400667208851
F1 medio:  0.7252308026509773
rendimiento:  0.263384786538729
número de instancias en el corpus:  14382


{'precision_global': 0.9482794650320423,
 'recall_global': 0.42824561955393375,
 'F1_global': 0.5900314226893488,
 'precision_mean': 0.9475733555833681,
 'recall_mean': 0.587400667208851,
 'F1_mean': 0.7252308026509773,
 'score': 0.263384786538729}

Vemos que la precisión es alta, lo que indica que lo que tiene que hacer el modelo (poner la primera letra en mayúscula y un punto al final), lo hace bien. Sin embargo, el recall es relativamente bajo ya que esos cambios no son suficientes para puntuar correctamente las oraciones, cosa que también se deduce del bajo rendimiento.

Aunque no se pide, definimos la función evaluate_example_from_corpus que permite ver la información resultante de evaluate_example para un elemento en concreto del corpus dado el número de línea corpus_line. De esta forma podemos explorar y verificar el correcto funcionamiento de las funciones definidas.

In [15]:
from evaluator import evaluate_example_from_corpus

In [16]:
corpus_line = 2
evaluate_example_from_corpus(addPunctuationBasic,check_file_path=CHECK_RAW_PATH,test_file_path=TEST_RAW_PATH,\
                             corpus_line=corpus_line)

TEST LINE: 
  people working in these canneries could barely stay there all day because of the smell but you know what they came out saying
MODEL PUNCTUATED LINE: 
  People working in these canneries could barely stay there all day because of the smell but you know what they came out saying.
VALIDATION LINE: 
  People working in these canneries could barely stay there all day because of the smell, but you know what they came out saying? 

Modificaciones necesarias:  {('S', 0), ('I', 23), ('I', 15)}
Modificaciones hechas por el modelo:  {('S', 0), ('I', 23)}
Diferencias entre modelo y validación:  {('S', 23), ('I', 15)} 

n_hechas (Núm. de modificaciones hechas por el modelo):  2
n_correctas (Núm. de modificaciones correctas, i.e., 
 interseccion(hechas,necesarias) - error de sustitucion de signo):  1
n_necesarias (Núm. de modificaciones necesarias):  3 

precision (n_correctas/n_hechas):  0.5
recall (n_correctas/n_necesarias):  0.3333333333333333 



## APARTADO 4. Modelo basado en 4-gramas. addPunctuation4gram.

Utilizando el corpus de entrenamiento contenido en PunctuationTask.train.en construimos un modelo de lenguaje inspirado en la idea de 4-gramas.

Definimos una función auxiliar *ngrams* que dado el string text devuelve la lista de N-gramas.

In [17]:
from preprocessor import ngrams

Veamos un ejemplo:

In [18]:
text_example = 'Sara said: Hello! nice to meet you!'

# Veamos los Ngramas con N de 2 a 4
for n in range(2,5):
    print(str(n)+'-gramas de la oración: ', text_example)
    print('\t', ngrams(text_example,n))

2-gramas de la oración:  Sara said: Hello! nice to meet you!
	 [('Sara', 'said'), ('said', ':'), (':', 'Hello'), ('Hello', '!'), ('!', 'nice'), ('nice', 'to'), ('to', 'meet'), ('meet', 'you'), ('you', '!')]
3-gramas de la oración:  Sara said: Hello! nice to meet you!
	 [('Sara', 'said', ':'), ('said', ':', 'Hello'), (':', 'Hello', '!'), ('Hello', '!', 'nice'), ('!', 'nice', 'to'), ('nice', 'to', 'meet'), ('to', 'meet', 'you'), ('meet', 'you', '!')]
4-gramas de la oración:  Sara said: Hello! nice to meet you!
	 [('Sara', 'said', ':', 'Hello'), ('said', ':', 'Hello', '!'), (':', 'Hello', '!', 'nice'), ('Hello', '!', 'nice', 'to'), ('!', 'nice', 'to', 'meet'), ('nice', 'to', 'meet', 'you'), ('to', 'meet', 'you', '!')]


Creamos la clase **ModelNgram**, que puede generalizarse fácilmente y que instanciaremos con N=4 para definir el modelo propuesto del apartado 4.

Esta clase tiene un método *entrena* para entrenar el modelo a partir de un archivo, un método *predice* que devuelve la operación más probable dada un N-1 tupla. Ademas como atributos cuenta con dos diccionarios:

 * **counts_dict**: para cada operación, cuenta con un diccionario donde cada clave es un N-1 tupla del corpus y su valor es el número de veces que aparece esa tupla precedida de la operación.
 * **trained_dict**: cada clave es una N-1 tupla del corpus de entrenamiento y su valor es la operación más probable (la que cuenta con mayorres repeticiones en count_dict)

In [19]:
from model_ngram import ModelNgram

Instanciamos el modelo y lo entrenamos.

In [20]:
model4gram = ModelNgram(N=4)
model4gram.entrena(train_file_path=TRAIN_RAW_PATH)

Consultamos el diccionario de conteo para comprobar la distribución de signos dada la terna ('by','the','way')

In [21]:
terna = ('by','the','way')
for i in model4gram.counts_dict:
    print('Ocurrencias con el signo ',i,' para la terna ',terna, ':',model4gram.counts_dict[i].get(terna,0))

Ocurrencias con el signo  <mayus>  para la terna  ('by', 'the', 'way') : 0
Ocurrencias con el signo  <minus>  para la terna  ('by', 'the', 'way') : 25
Ocurrencias con el signo  .  para la terna  ('by', 'the', 'way') : 36
Ocurrencias con el signo  ,  para la terna  ('by', 'the', 'way') : 200
Ocurrencias con el signo  ;  para la terna  ('by', 'the', 'way') : 1
Ocurrencias con el signo  :  para la terna  ('by', 'the', 'way') : 0
Ocurrencias con el signo  ?  para la terna  ('by', 'the', 'way') : 1
Ocurrencias con el signo  !  para la terna  ('by', 'the', 'way') : 0


Se observa que la mayor ocurrencia se da para la coma ',' por lo que la función *predice* del modelo debe devolver dicho caracter.

In [22]:
print('La operación más probable dada la terna ',terna, ' es: ',model4gram.predice(terna))

La operación más probable dada la terna  ('by', 'the', 'way')  es:  ,


Igual que definimos el modelo de forma genérica para Ngramas, vamos a definir la función genérica addPunctuationNgram donde la función requerida por el apartado addPunctuation4gram es addPunctuationNgram usando un modelo basado en 4 gramas.

Esta función cuenta con un parámetro *add_punc_basic* que añade puntuación básica como la de addPuntuationBasic ya que el modelo de 4 gramas nunca va a poner la primera letra en mayúsculas y puede que deje la oración sin puntuación final.

In [23]:
from punctuator_ngram import addPunctuationNgram

Podemos definir ahora la función de puntuación addPunctuation4gram simplemente como la ejecución de addPunctuationNgram
verificando que el modelo usado usa 4gramas.

In [24]:
def addPunctuation4gram(model,example,add_basic_punct = False):
    N = model.N
    assert N == 4, 'The model is based in {} -grams and it should be 4-grams.'.format(str(N))
    return addPunctuationNgram(model,example,add_basic_punct = add_basic_punct)


In [25]:
text_example = "and we also are eating meat that comes from some of these same places"
print('Frase sin puntuar:\n \t',text_example)
print('Puntuación de la frase con modelo 4 gramas:\n \t',addPunctuation4gram(model4gram,text_example,add_basic_punct=False))
print('Puntuación de la frase con modelo 4 gramas + addPunctuationBasic:\n \t',addPunctuation4gram(model4gram,text_example,add_basic_punct=True))

Frase sin puntuar:
 	 and we also are eating meat that comes from some of these same places
Puntuación de la frase con modelo 4 gramas:
 	 and we also are eating meat, that comes from some of these same places
Puntuación de la frase con modelo 4 gramas + addPunctuationBasic:
 	 And we also are eating meat, that comes from some of these same places.


Exploremos algunos ejemplos usando puntuación básica y sin ella en el modelo de 4gramas.

In [27]:
# Instancia del corpus. Modificar para ver distintos ejemplos.
i = 0

l = 70
print('='*l)
print('MODELO 4GRAMS')
print('='*l)
evaluate_example_from_corpus(addPunctuation4gram, model = model4gram,test_file_path=TEST_RAW_PATH,\
                             check_file_path= CHECK_RAW_PATH, add_punct_basic=False,corpus_line = i)
print('='*l)
print('='*l)
print('4GRAMS + PUNTUACION BÁSICA')
print('='*l)
evaluate_example_from_corpus(addPunctuation4gram, model = model4gram, test_file_path=TEST_RAW_PATH,\
                             check_file_path= CHECK_RAW_PATH,add_punct_basic=True,corpus_line = i)
print('='*l)

MODELO 4GRAMS
TEST LINE: 
  it can be a very complicated thing the ocean
MODEL PUNCTUATED LINE: 
  it can be a very complicated thing, the ocean
VALIDATION LINE: 
  It can be a very complicated thing, the ocean. 

Modificaciones necesarias:  {('S', 0), ('I', 7), ('I', 9)}
Modificaciones hechas por el modelo:  {('I', 7)}
Diferencias entre modelo y validación:  {('S', 0), ('I', 10)} 

n_hechas (Núm. de modificaciones hechas por el modelo):  1
n_correctas (Núm. de modificaciones correctas, i.e., 
 interseccion(hechas,necesarias) - error de sustitucion de signo):  1
n_necesarias (Núm. de modificaciones necesarias):  3 

precision (n_correctas/n_hechas):  1.0
recall (n_correctas/n_necesarias):  0.3333333333333333 

4GRAMS + PUNTUACION BÁSICA
TEST LINE: 
  it can be a very complicated thing the ocean
MODEL PUNCTUATED LINE: 
  It can be a very complicated thing, the ocean.
VALIDATION LINE: 
  It can be a very complicated thing, the ocean. 

Modificaciones necesarias:  {('S', 0), ('I', 7), ('I

## APARTADO 5: Evaluación de addPunctuation4gram y comparación de modelos.

Evaluemos el modelo usando la misma función *evaluate* anterior. Vamos a comparar el rendimiento usando también la puntuación básica y sin ella.

In [28]:
print('MODELO 4GRAMS')
evaluate(addPunctuation4gram,model=model4gram,test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH)
print()

MODELO 4GRAMS
MÉTRICAS
precision global:  0.3061919504643963
recall global:  0.04666708609896505
F1 global:  0.08099033684555332
precision media:  0.14064391487397296
recall medio:  0.04956605113889393
F1 medio:  0.07329966587077798
rendimiento:  0.0
número de instancias en el corpus:  14382



Podemos ver que este modelo tal cual es muy pobre. Usando la función *evaluate_example_from_corpus*, podemos ver que nunca pone la mayúscula inicial como era de esperar y rara vez un punto al final. Además las predicciones parecen bastante pobres y esto se puede deber a la falta de variedad en en las tuplas del corpus de entrenamiento. Una posible mejoría se conseguiría con un corpus más grande y variado.

Veamos la evaluación añadiendo la puntuación básica y lo comparamos con lo obtenido en addPunctuationBasic.

In [29]:
print('4GRAMS + PUNTUACION BÁSICA')
evaluate(addPunctuation4gram,model=model4gram,test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH,\
         add_punct_basic=True)
print()
print('PUNTUACIÓN BÁSICA')
evaluate(addPunctuationBasic,test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH,)
print()

4GRAMS + PUNTUACION BÁSICA
MÉTRICAS
precision global:  0.7801183367117734
recall global:  0.447937965963069
F1 global:  0.569101954358339
precision media:  0.8433161902915692
recall medio:  0.5991575233871246
F1 medio:  0.7005732377871406
rendimiento:  0.2369628702544848
número de instancias en el corpus:  14382

PUNTUACIÓN BÁSICA
MÉTRICAS
precision global:  0.9482794650320423
recall global:  0.42824561955393375
F1 global:  0.5900314226893488
precision media:  0.9475733555833681
recall medio:  0.587400667208851
F1 medio:  0.7252308026509773
rendimiento:  0.263384786538729
número de instancias en el corpus:  14382



Podemos ver que en general la precisión es más baja en el modelo de 4gramas pero el recall ligeramente superior, pero no de forma sustancial. Además los F1 son menores en el modelo de puntuación básica que en el de 4 gramas así como el rendimiento.

En general podemos concluir que el modelo de puntuación básica es ligeramente mejor que el basado en 4gramas cuando se añade la puntuación línea a línea por sorprendente que parezca. Esto puede deverse a que muchas de las oraciones del corpus de entrenamiento no presentan signos intermedios (sólo mayúscula inicial y punto final, exactamente lo que hace addPunctuationBasic)

Ya que hemos implementado un modelo genérico, vamos a comparar con modelos basados en 3gramas y 5gramas para ver si existe mejoría o no.

In [32]:
model3gram = ModelNgram(N=3)
model5gram = ModelNgram(N=5)
model3gram.entrena(train_file_path=TRAIN_RAW_PATH)
model5gram.entrena(train_file_path=TRAIN_RAW_PATH)

In [33]:
print('3GRAMAS')
evaluate(addPunctuationNgram, model = model3gram,test_file_path=TEST_RAW_PATH,\
         check_file_path= CHECK_RAW_PATH, add_punct_basic=True)

3GRAMAS
MÉTRICAS
precision global:  0.7269191453289785
recall global:  0.45858630343829626
F1 global:  0.5623848698486792
precision media:  0.8041213477542051
recall medio:  0.6055814495149251
F1 medio:  0.6908704051694091
rendimiento:  0.2259769155889306
número de instancias en el corpus:  14382


{'precision_global': 0.7269191453289785,
 'recall_global': 0.45858630343829626,
 'F1_global': 0.5623848698486792,
 'precision_mean': 0.8041213477542051,
 'recall_mean': 0.6055814495149251,
 'F1_mean': 0.6908704051694091,
 'score': 0.2259769155889306}

In [34]:
print('5GRAMAS')
evaluate(addPunctuationNgram, model = model5gram, test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH,add_punct_basic=True)
print()

5GRAMAS
MÉTRICAS
precision global:  0.8715371568565865
recall global:  0.43644027808361385
F1 global:  0.581621530980129
precision media:  0.9022099935324751
recall medio:  0.592608753166226
F1 medio:  0.7153476507331434
rendimiento:  0.2533027395355305
número de instancias en el corpus:  14382



Observamos que en cuanto a precision, el modelo basado en 3gramas es ligeramente inferior al de 4gramas y este a su vez inferior al de 5 gramas, todos por debajo de la precisión del puntuador básico. En cuanto a recall el orden es inverso: el modelo basado en 3 gramas presenta mayor recall que el de 4 y este a su vez que el de 5.
En cuanto al F1, los mayores valores los encontramos para el modelo de 5gramas y de puntuación básica. Es obvio que cuanto mayor sea N, menos probable es la probabilidad de que se de una N-tupla concreta por lo que el modelo basado en N-gramas con N grande coincidirá eventualmente con el de puntuación básica si se considera el parámetro add_punct_basic.

Para acabar este apartado podemos explorar algunas predicciones de los modelos de 3 y 5 gramas:

In [35]:
# Instancia del corpus. Modificar para ver distintos ejemplos.
i= 0

l = 70 
print('='*l)
print('MODELO 3GRAMS')
print('='*l)
evaluate_example_from_corpus(addPunctuationNgram, model = model3gram,test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH, add_punct_basic=True,corpus_line = i)
print('='*l)
print('MODELO 5GRAMS')
print('='*l)
evaluate_example_from_corpus(addPunctuationNgram, model = model5gram,test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH, add_punct_basic=True,corpus_line = i)

MODELO 3GRAMS
TEST LINE: 
  it can be a very complicated thing the ocean
MODEL PUNCTUATED LINE: 
  It can be a very complicated thing. The ocean.
VALIDATION LINE: 
  It can be a very complicated thing, the ocean. 

Modificaciones necesarias:  {('S', 0), ('I', 7), ('I', 9)}
Modificaciones hechas por el modelo:  {('S', 0), ('S', 7), ('I', 7), ('I', 9)}
Diferencias entre modelo y validación:  {('S', 7), ('S', 8)} 

n_hechas (Núm. de modificaciones hechas por el modelo):  4
n_correctas (Núm. de modificaciones correctas, i.e., 
 interseccion(hechas,necesarias) - error de sustitucion de signo):  2
n_necesarias (Núm. de modificaciones necesarias):  3 

precision (n_correctas/n_hechas):  0.5
recall (n_correctas/n_necesarias):  0.6666666666666666 

MODELO 5GRAMS
TEST LINE: 
  it can be a very complicated thing the ocean
MODEL PUNCTUATED LINE: 
  It can be a very complicated thing, the ocean.
VALIDATION LINE: 
  It can be a very complicated thing, the ocean. 

Modificaciones necesarias:  {('S', 

Fin de este documento. El apartado 6 se puede seguir en el siguiente Notebook. Esta separación se debe a dos cosas:
 * La distinta naturaleza de los apartados (implementación y desarrollo frente a exploración y adaptación)
 * El notebook del apartado 6 está pensado para ser ejecutado a parte en Google Collab con GPU ya que el modelo tarda mucho en entrenar y puntuar.